<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Не-BERT-модели" data-toc-modified-id="Не-BERT-модели-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Не BERT модели</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>BERT</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Классификация комментариев c BERT

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучим модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

---

**Шаги выполнения проекта**

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделайем выводы.

---

**Описание данных**

Данные находятся в файле `toxic_comments.csv`.


Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем библиотеки

In [1]:
# импорты из стандартной библиотеки
import re
import warnings

# импорты сторонних библиотек
import pandas as pd
import numpy as np

# импорты модулей текущего проекта
import nltk
import spacy
import torch
from transformers import (
    AutoTokenizer,
    TextClassificationPipeline,
    AutoModelForSequenceClassification,
    BertTokenizer,
    BertConfig,
    BertModel
)
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
    RandomizedSearchCV
)
from sklearn.linear_model import (
    LogisticRegression, 
    RidgeClassifier,
    SGDClassifier
)
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from tqdm import tqdm, notebook

# настройки
warnings.filterwarnings("ignore")
tqdm.pandas(desc='progress')

# константы заглавными буквами
RANDOM_STATE = 12345

### Обзор данных

Считаем файл toxic_comments.csv

In [2]:
try:
    toxic_comments = pd.read_csv('toxic_comments.csv')
except:
    toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
toxic_comments.tail()

,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\nAnd ... I really don't think you understand...",0


In [4]:
# сразу дропнем ненужный столбец
toxic_comments = toxic_comments.drop('Unnamed: 0', axis=1)

In [5]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
toxic_comments.describe(include='all')

,text,toxic
count,159292,159292.000000
unique,159292,NaN
top,"""\n\n Consonant and Vowel inventory?? \n\nHey ...",NaN
freq,1,NaN
mean,NaN,0.101612
std,NaN,0.302139
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,0.000000


In [7]:
# есть дисбаланс
toxic_comments['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Кроме дисбаланса проблем не вижу, все нормально.

### Подготовка данных

Попробуем использовать spacy для лемматизации (pymystem работал очень долго и я от него отказался).

Объявим функции lemmatize() и clear()

In [8]:
def lemmatize(text):  
    # лемматизируем текст используя модуль nlp
    doc = nlp(text)
    lemm_text = ' '.join([token.lemma_ for token in doc]).lower()
    
    return lemm_text


def clear_text(lemm_text):
    # чистим текст
    cleared_text = re.sub(r'[^a-zA-Z\']', ' ', lemm_text).split()
    cleared_text = ' '.join(cleared_text)
    
    return cleared_text

Пробуем скачать файл с лемматизированными данными, если его нет, лемматизируем текст и сохраняем файлик.


Также распараллелим процесс, чтобы лемматизация прошла быстрее.

In [9]:
# грузим модуль для обработки текста
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [10]:
# применим progress_apply
try:
    toxic_comments = pd.read_csv('toxic_lemm.csv', index_col=[0])
except:
    toxic_comments['lemm_text'] = toxic_comments['text'].progress_apply(lambda x: clear_text(lemmatize(x)))
    toxic_comments.to_csv('toxic_lemm.csv')

**Промежуточный вывод**:

- данные загружены и изучены
- дропнут лишний столбец
- удалены строки с кириллицей
- объявили функции lemmatize и clear text
- лемматизировали и очистили текст

## Обучение

Разделим датасет на train и test выборку

In [11]:
toxic_comments.isna().sum()

text          0
toxic         0
lemm_text    10
dtype: int64

In [12]:
# избавимся от 10 пропусков
toxic_comments = toxic_comments[~toxic_comments['lemm_text'].isna()].reset_index(drop=True)

In [13]:
# Обозначим признак и таргет
X = toxic_comments['lemm_text']
y = toxic_comments['toxic']

# сохраним отношение классов stratify=y
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=RANDOM_STATE)

In [14]:
# ОК
y_train.value_counts(normalize=True)

0    0.898323
1    0.101677
Name: toxic, dtype: float64

In [15]:
# качаем стоп слова
nltk.download('stopwords')

# содаем лист стоп слов на английском
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nikolaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# учитывая стоп-слова считаем TF-IDF для трейн и тест корпуса
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
tf_idf_train = count_tf_idf.fit_transform(X_train)
tf_idf_test = count_tf_idf.transform(X_test)

# выведем размеры матриц
print("Размер трейн матрицы:", tf_idf_train.shape)
print("Размер тест матрицы:", tf_idf_test.shape)

Размер трейн матрицы: (143120, 147217)
Размер тест матрицы: (15903, 147217)


### Не BERT модели

Пробуем получить cv_score без использования пайплайна

In [17]:
%%time

# создаем эстиматор
lr_model = LogisticRegression(class_weight='balanced')

# считаем cv f1-score
lr_cv_score = cross_val_score(lr_model, tf_idf_train, y_train, scoring='f1', cv=5).sum() / 5

print(f'LR TF-IDF F1 CV SCORE: {lr_cv_score:.2f}')

LR TF-IDF F1 CV SCORE: 0.75
Wall time: 14.6 s


А теперь попробуем с применением пайплайна

In [18]:
%%time

# пишем пайплайн для логистической регрессии
lr_pipeline = Pipeline(
    [
        ("tfidf", TfidfVectorizer(stop_words=stop_words)),
        ("clf", LogisticRegression(class_weight='balanced')),
    ]
)

# считаем cv f1-score
lr_cv_score = cross_val_score(lr_pipeline, X_train, y_train, scoring='f1', cv=5).sum() / 5
print(f'LR TF-IDF F1 CV SCORE: {lr_cv_score:.2f}')

LR TF-IDF F1 CV SCORE: 0.75
Wall time: 53.4 s


Существенной разницы нет, но, мы избавились от утечки данных, это хорошо.


Теперь используем пайплайн с gridsearch, попробуем подобрать гиперпараметр C

In [19]:
%%time

lr_grid_params = {'clf__C': range(5, 16, 1)}

lr_grid = GridSearchCV(
    lr_pipeline, param_grid=lr_grid_params, 
    scoring='f1', n_jobs=-1, cv=5
)

lr_grid.fit(X_train, y_train)

print(f'LR GRID BEST PARAMS: {lr_grid.best_params_}')
print(f'LR GRID BEST SCORE: {lr_grid.best_score_:.2f}')
print('---------------------------')

LR GRID BEST PARAMS: {'clf__C': 6}
LR GRID BEST SCORE: 0.77
---------------------------
Wall time: 2min 19s


Неплохо!

Попробовал также и модели RidgeClassifier и SGDClassifier

In [20]:
# создание конвейера для обработки текстовых данных и классификации
pipeline = Pipeline(
    [
        ("tfidf", TfidfVectorizer(stop_words=stop_words)),
        ("clf", RidgeClassifier()),
    ]
)

# задание сетки гиперпараметров для Ridge и SGDC
parameters = {
    "clf": [
        RidgeClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        SGDClassifier(
            loss="log_loss", class_weight="balanced", random_state=RANDOM_STATE
        ),
    ],
    "clf__alpha": [0.001, 0.01, 0.1, 1, 10],
    "clf__max_iter": [1000, 5000, 10000],
    "clf__tol": [1e-3, 1e-4, 1e-5]
}

# создание объекта GridSearchCV для подбора гиперпараметров
randomized_search = RandomizedSearchCV(pipeline, parameters, cv=5, n_jobs=-1, scoring='f1')

# запуск поиска по сетке
randomized_search.fit(X_train, y_train)

# вывод лучших гиперпараметров и оценки качества
print(f"Лучшие гиперпараметры: {randomized_search.best_params_}")
print(f"Оценка качества: {randomized_search.best_score_}")

Лучшие гиперпараметры: {'clf__tol': 0.001, 'clf__max_iter': 1000, 'clf__alpha': 1, 'clf': RidgeClassifier(alpha=1, class_weight='balanced', max_iter=1000,
                random_state=12345, tol=0.001)}
Оценка качества: 0.6982359588723536


Оценка вышла не лучшая

Пробуем использовать пайплайн и для lgbm классификатора

In [21]:
%%time

# объявляем пайплайн для lgbm
lgbm_pipeline = Pipeline(
    [
        ("tfidf", TfidfVectorizer(stop_words=stop_words)),
        ("clf", LGBMClassifier(
            class_weight='balanced', n_jobs=-1, 
            random_state=RANDOM_STATE)
        ),
    ]
)

# считаем cv f1-score
lgbm_cv_score = cross_val_score(lgbm_pipeline, X_train, y_train, scoring='f1', cv=5, n_jobs=-1).sum() / 5
print(f'LGBM TF-IDF F1 CV SCORE: {lgbm_cv_score:.2f}')

LGBM TF-IDF F1 CV SCORE: 0.74
Wall time: 1min 7s


### BERT

Попробуем применить обычную модель BERT без тюнинга, для начала.

In [22]:
%%time

# задаем путь претренированной модели и device 
model_path = 'unitary/toxic-bert'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# создаем токенайзер + модель
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)

# создаем семпл из 1000 строк
test_bert = toxic_comments.sample(1000, weights='toxic', random_state=RANDOM_STATE)

# предиктим токсичные комментарии
y_pred = []
for text in tqdm(test_bert['text']):
    tokenized = tokenizer.encode( text, truncation=True, max_length=512, add_special_tokens=True)
    tokenized = np.array(tokenized)
    attention_mask = np.where(tokenized != 0, 1, 0)
    batch = torch.tensor([tokenized]).to(device) 
    attention_mask_batch = torch.tensor([attention_mask]).to(device)

    pred = model(batch, attention_mask_batch)

    y_pred.append(1 if bool(pred[0][0][0].cpu().detach() > 0) else 0)
    
# считаем скор и выводим на экран
bert_f1_score = f1_score(test_bert['toxic'], y_pred)
print(f'BERT F1-SCORE ON SAMPLE: {bert_f1_score:.2f}')

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:02<00:00,  5.49it/s]

BERT F1-SCORE ON SAMPLE: 0.96
Wall time: 3min 17s


На сэмпле из 10000 значений BERT показал себя отлично, на полном наборе данных, думаю, можно ожидать похожей оценки.

---

Быстрый тест LogisticRegression на test

In [23]:
%%time

best_model = lr_grid.best_estimator_
best_model.fit(X_train, y_train)
best_model_pred = best_model.predict(X_test)
best_model_f1_score = f1_score(y_test, best_model_pred)

print(f'LR BEST MODEL F1-SCORE: {best_model_f1_score:.3f}')

LR BEST MODEL F1-SCORE: 0.763
Wall time: 12.4 s


**Промежуточный вывод**:

- избавились от пропусков после лемматизации
- разбили датасет на train и test, учли дисбаланс
- учитывая стоп-слова посчитали TF-IDF для train и test корпуса
- обучили *LogisticRegression* (F1-score: 0.77) и *LGBMClassifier* (F1-score: 0.74)
- *BERT* на семпле из 1000 строк показал себя **отлично**, F1-score: 0.96
---
- на тесте LogisticRegression c подбором гиперпараметра C показала F1-score: 0.763

## Выводы

**`Общий вывод`**:


- **Подготовка**:

    - данные загружены и изучены
    - дропнут лишний столбец
    - удалены строки с кириллицей
    - объявили функции lemmatize и clear text
    - лемматизировали и очистили текст


- **Обучение**:

    - избавились от пропусков после лемматизации
    - разбили датасет на train и test, учли дисбаланс
    - учитывая стоп-слова посчитали TF-IDF для train и test корпуса
    - обучили *LogisticRegression* (F1-score: 0.77) и *LGBMClassifier* (F1-score: 0.74)
    - *BERT* на семпле из 1000 строк показал себя **отлично**, F1-score: 0.76
---
   - на тесте LogisticRegression c подбором гиперпараметра C показала F1-score: 0.763